#### 程序清单6-3 完整版的Platt SMO的支持函数

In [1]:
import numpy as np


class OptStruct:
    def __init__(self,dataMatin,classLabels,C,toler):
        self.X=dataMatin
        self.labelMat=classLabels
        self.C=C
        self.tol=toler
        self.m=np.shape(dataMatin)[0]
        self.alphas=np.mat(np.zeros((self.m,1)))
        self.b=0
        self.eCache=np.mat(np.zeros((self.m,2)))    # 第一列表示标识位是否有效
        

def calcEk(oS,k):
    """计算第k个样本的误差"""
    fXk=float(np.multiply(oS.alphas,oS.labelMat).T*(oS.X*oS.X[k,:].T))+oS.b
    Ek=fXk-float(oS.labelMat[k])
    return Ek


def selectJ(i,oS,Ei):
    """先验知识选择J
    选择能使误差改变最大的那个J"""
    
    # 初始化
    maxK=-1
    maxDeltaE=0
    Ej=0
    
    oS.eCache[i]=[1,Ei]
    # 构建一个非零表
    vaildEcacheList=np.nonzero(oS.eCache[:,0].A)[0]    # 返回一个元组，元组的第一个元素是索引列表
    if len(vaildEcacheList)>1:
        for k in vaildEcacheList:
            if k==i:
                continue
            Ek=calcEk(oS,k)    # 计算误差
            deltaE=np.abs(Ei-Ek)
            if deltaE>maxDeltaE:
                maxDeltaE=deltaE    # 更新最大误差变化量
                maxK=k
                Ej=Ek
    return maxK,Ej


def updataEk(oS,k):
    """计算误差值,并存入缓存"""
    Ek=calcEk(oS,k)
    oS.eCache[k]=[1,Ek]
    

def clipAlpha(aj, H, L):
    """
    防止选择到过大或者过小的aj,合并用得函数,相当于np.clip()
    aj:输入的数
    H:上边界
    L:下边界
    """
    if aj > H:
        return H
    if aj < L:
        return L
    return aj


#### 程序清单6-4 完整Platt SMO算法中的优化例程

In [ ]:
def innerL(i,oS):
    Ei=calcEk(i,oS)
    if ((oS.labelMat[i]*Ei<-oS.tol) and (oS.alphas[i]<oS.C)) or\
        ((oS.labelMat[i]*Ei>oS.tol) and (oS.alphas[i]>0)):
        j,Ej=selectJ(i,oS,Ei)
        alphaIold=oS.alphas[i].copy()
        alphaJold=oS.alphas[j].copy()
        
        if oS.labelMat[i]!=oS.labelMat[j]:
            L=max(0,oS.alphas[j]-oS.alphas[i])
            H=min(oS.C,oS.C+oS.alphas[j]-oS.alphas[i])
        else:
            L=max(0,oS.alphas[j]+oS.alphas[i]-oS.C)
            H=min(oS.C,oS.alphas[i]+oS.alphas[j])
        
        if L==H:
            print('L==H!')
            return 0   # 利用return语句结束循环
        
        eta=2*oS.X[i,:]*oS.X[j,:].T-oS.X[i,:]*oS.X[i,:].T-oS.X[j,:]*oS.X[j,:].T
        if eta>=0:
            print('eta>=0!')
            return 0
        # 更新alpha_j
        oS.alphas[j]-=oS.labelMat[j]*(Ei-Ej)/eta
        oS.alphas[j]=clipAlpha(oS.alphas[j],H,L)
        updataEk(oS,j)
        
        if np.abs(oS.alphas[j]-alphaJold)<0.00001:
            print('j is not moving enough!')
            return 0
        
        # 更新alpha_i
        oS.alphas[i]+=oS.labelMat[j]*oS.labelMat[i]*(alphaJold-oS.alphas[j])
        updataEk(oS,i)
        
        # 计算常数项b
        b1 = oS.b-Ei-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i, :]*oS.X[i, :].T *\
            oS.labelMat[j]*(oS.alphas[j]-alphaJold) * \
            oS.X[i, :]*oS.X[j, :].T
        b2 = oS.b-Ej-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i, :]*oS.X[j, :].T *\
            oS.labelMat[j]*(oS.alphas[j]-alphaJold) * \
            oS.X[j, :]*oS.X[j, :].T

        if (0<oS.alphas[i]) and (oS.C>oS.alphas[i]):
            oS.b=b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b=(b1+b2)/2.0
            
        return 1
    
    else:
        return 0
    